<a href="https://colab.research.google.com/github/danperazzo/computational_creativity/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Tutorial do Grupo Separatistas

Licensed under the Apache License, Version 2.0

In [ ]:
# Copyright 2020 Daniel Perazzo, Leão Liu, Nara Andrade, Mayara Nunes
# Mario Wassen, Aline Gouveia e Rebeca Silva 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feature Visualization for Neural Networks

In the following tutorial we will use Feature Visualization techniques to create really cool abstract images. For this, we will use the [lucid visualition library](https://github.com/tensorflow/lucid) coupled with [Tensorflow at version 1.15](https://www.tensorflow.org/?hl=pt-br)

![](https://storage.googleapis.com/tensorflow-lucid/notebooks/xy2rgb/cppn-header.jpg)
This tutorial will teach you how to use Feature Visualizations for creating really cool abstract images. 

This tutorial is heavily based on the colab notebook provided in [this link](https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/differentiable-parameterizations/xy2rgb.ipynb) and also the multiple lucid notebooks available on their repo (links at the end of the tutorial). Thanks for the authors for sharing the code with the Apache 2.0 License. If you want to know more about this topic, we recommend you to check out [this repo](https://github.com/tensorflow/lucid).




## Introduction: WTF are these things?

First off. What the hell are Feature Visualizations? 
To respond, let's imagine a classifier. We can imagine that the neurons fire up to detect the objects they are looking for. But, what are the neurons seeing? Why are they firing in such a way? 

Feature visualizations let us see what estimulates each neuron. The idea is, instead of optimizing the neural network's weights during to accomodate the input, in here we optimized the image to maximize the neuron's activation. In this tutorial we will see some ideas and create some cool visualizations.


So in, the following section, we will see some neurons firing up and, finally, create some trippy images. 

## Install, Import, and load a model

In here we will just import and install the necessary modules, including Tensorflow and Lucid

In [ ]:
!pip uninstall numpy # You will have to type n
!pip install numpy==1.16.1
!pip install tf_slim
!pip install -q lucid>=0.2.3

Now we will just import the necessary packages for our tutorial

In [ ]:
import io
import numpy as np
import string
import PIL
import base64
from glob import glob
import scipy.ndimage as nd

import matplotlib.pylab as pl

%tensorflow_version 1.x # Just in case... Lucid only works for Tensorflow == 1.x
import tensorflow as tf
import tf_slim as slim

from IPython.display import clear_output, Image, display, HTML

from google.colab import files

In [ ]:
from lucid.modelzoo import vision_models
from lucid.misc.io import show, save, load
from lucid.optvis import objectives
from lucid.optvis import transform
from lucid.optvis import param
from lucid.optvis import render
from lucid.misc.tfutil import create_session
from lucid.recipes import caricature

## Getting Started: Neural visualizations

In this module, we will just be playing around and creating some images. To do this, we will optimize an image to fire up some neurons as much as possible. First, we will need to load a model just to see what we are talking about.

In [ ]:
model = vision_models.InceptionV1()
model.load_graphdef()

Now, with the model loaded, let's visualize what is happenning on channel 10 of layer "mixed4a_pre_relu" 

In [ ]:
obj = objectives.channel("mixed4a_pre_relu", 465)
_ = render.render_vis(model, obj)

We can try different layers channels, in particular, we can see that if we put shallower layers, the images get simpler, as expected given how ConNets work. The output of this code is one of the things that you should deliver at the end of this tutorial.

In [ ]:
# Just set the layer and channel like the previous example, changing the layer
# for 'conv2d2' and the channel to 10
###########YOUR CODE HERE################# 
obj = None
###########YOUR CODE HERE################# 
_ = render.render_vis(model, obj)


Cool, we can also inspect how a specific neuron (ie: a specific x,y position on a specific layer on a specific channel) is estimulated

In [ ]:
obj = objectives.neuron("conv2d2", 10,x=10,y=10)
_ = render.render_vis(model, obj)

## Some cool things you can do

On the introductory side, we can do some cool things with feature visualizations:

In [ ]:
# We could optimize two neurons at the same time

obj =objectives.channel("mixed4a_pre_relu", 476) + objectives.channel("mixed4a_pre_relu", 465)
_ = render.render_vis(model, obj)

In [ ]:
# We could see two images: on the left we see the image that estimulates less the neuron. 
# On the right the image that estimulates the neuron more.

param_f = lambda: param.image(128, batch=2)
obj = objectives.channel("mixed4a_pre_relu", 492, batch=1) - objectives.channel("mixed4a_pre_relu", 492, batch=0)
_ = render.render_vis(model, obj, param_f)

In [ ]:
#Finally, using techniques better explained on here: https://distill.pub/2017/feature-visualization/
# We can better see this neuron firing on a diversity of settings

param_f = lambda: param.image(128, batch=6)
obj = obj = objectives.channel("mixed5a_pre_relu", 9) - 1e2*objectives.diversity("mixed5a")
_ = render.render_vis(model, obj, param_f)

## Getting started with Differentiable Image Parametrization: CPPNs

Well we are over with the preliminaries, and let's see what we can do. Differentiable Image Parametrization is just a really cool way to parametize an image so that it is differentiable. In particular,in this section we will see how we can use Compositional Pattern Producing Networks(CPPNs) to create trippy images. 

CPPNs are just a network whose weights parametrize an image. Our idea is to optimize the weights so that our specified CNN channels gets as activated as possible. In doing this, we will create some trully great images. 

In [ ]:
# Let's load the CPPN fuction from lucid
image_cppn = param.cppn

In here we will again load a model. In a few moments you will play around with some of these parameters

In [ ]:
################HERE YOU WILL PLAY AROUND#################################
#Test which models will go here, we recommend using the models: InceptionV1,
model = vision_models.InceptionV1()
#model = vision_models.VGG19_caffe()
#model = vision_models.AlexNet()
################HERE YOU WILL PLAY AROUND#################################
model.load_graphdef()

Just to give you a sense of how BIG this module is, it has exactly:

In [ ]:
with tf.Graph().as_default():
  image_cppn(224)
  variables = tf.get_collection('variables')
  param_n = sum([v.shape.num_elements() for v in variables])
  print('This amount of parameters:', param_n)

This amount of parameters: 8451


Now, let's go on and perform feature visualizations. The fun starts now! We will test our feature visualization framework with more examples. Here you will alter the neurons and different models, bellow is a list of images for you to test. Just substitute the string_obj for the required string.

If you are testing on InceptionV1Net, try -> ''conv2d2', 'mixed4a_3x3_pre_relu' and 'head0_bottleneck'

If you are testing on VGG19-, try -> 'conv3_3/conv3_3', 'conv5_3/conv5_3' and
'fc7/fc7'

If you are testing on AlexNet-, try -> 'Conv2D_2', 'Conv2D_5' and
'Conv2D_7'

To see the full list of other node names for these three models, or any other model you might want to check [this link](https://github.com/tensorflow/lucid/tree/master/lucid/modelzoo). 
As expected the deeper we get our neurons the more complex and interesting the images get.


In [ ]:
###############MODIFY HERE ###############
string_obj= None
###############MODIFY HERE ###############

def render_feature(
    cppn_f = lambda: image_cppn(224),
    optimizer = tf.train.AdamOptimizer(0.005),
    objective = objectives.channel(string_obj, 20)):
  vis = render.render_vis(model, objective, param_f=cppn_f, optimizer=optimizer, transforms=[], thresholds=[2**i for i in range(5,10)], verbose=False)
  show(vis)

render_feature()

## Deep Caricature generation

Well, now let's create dreamy versions of already existing objects. To do this, let's use Feature Inversions and use [these ideas](https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/misc/feature_inversion_caricatures.ipynb#scrollTo=RDCKv_xfviPQ). What we are going to do is similar to what is done with DeepDream. We will load the model and start the definitions.

In [ ]:
# Import the InceptionV1 (GoogLeNet) model from the Lucid modelzoo

model = vision_models.InceptionV1()
model.load_graphdef()

In [ ]:
# Now we will just get the caricature function from the lucid package
feature_inversion = caricature.feature_inversion

Now, let's just choose some layers and see what their neurons are seeing when they look at the image.

In [ ]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png")

layers = ['conv2d%d' % i for i in range(0, 3)] + ['mixed3a', 'mixed3b', 
                                                  'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e',
                                                  'mixed5a', 'mixed5b']

In [ ]:
for layer in layers:
  print(layer)
  caricature.feature_inversion(img, model,layer=layer)
  print("")

Just for the fun, you can now just choose some image, preferentially your photo, and upload to see how some neuron sees you.

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  img = load(fn)
  show(imgToModelSize(img))
  feature_inversion(img, layer='mixed4d')

##Goodbye


Sadly this really small tutorial just ended. However, you can check out these awesome links:

###Repositories
* [Lucid](https://github.com/tensorflow/lucid/)

###Recomended Reading

* [Feature Visualization](https://distill.pub/2017/feature-visualization/)
* [The Building Blocks of Interpretability](https://distill.pub/2018/building-blocks/)
* [Using Artiﬁcial Intelligence to Augment Human Intelligence](https://distill.pub/2017/aia/)
* [Visualizing Representations: Deep Learning and Human Beings](http://colah.github.io/posts/2015-01-Visualizing-Representations/)
* [Differentiable Image Parameterizations](https://distill.pub/2018/differentiable-parameterizations/)
* [Activation Atlas](https://distill.pub/2019/activation-atlas/)

### Related Talks
* [Lessons from a year of Distill ML Research](https://www.youtube.com/watch?v=jlZsgUZaIyY) (Shan Carter, OpenVisConf)
* [Machine Learning for Visualization](https://www.youtube.com/watch?v=6n-kCYn0zxU) (Ian Johnson, OpenVisConf)

Since we have to grade you, we ask you to submit the images generated varying the parameters of the CPPN. And submit also the image that you used to generate the DeepDreamish images.